<a href="https://colab.research.google.com/github/LeoLelli/AI_KialoProject/blob/main/KialoProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kialo Project

**Goal**: to help a user to join a Kialo discussion

**Approach**: analyze all comments and provide the user with the one with the highest semantic similarity to the user's comment

I use a transformer to do embedding of comments and then cosin similarity to calculate their similarity score. The similarity score indicates whether two texts have similar or more different meanings. 

In [13]:
#!pip install sentence-transformers
#import
import re
import sys
from sentence_transformers import SentenceTransformer, util
import numpy as np
 
%matplotlib inline
input_file = "ps5-v-xbox-series-xs-whats-better-48036.txt"

# pre-trained transformers: https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0
model = SentenceTransformer('stsb-distilroberta-base-v2')

In [14]:
class Comment:
  def __init__(self, level, content, sentiment):
    self.level = level
    self.content = content
    self.sentiment = sentiment

  def __str__(self):
        return self.level + " " + self.content +" "+ "["+self.sentiment+"]"
  
  def setSimilarity(self, similarity):
        self.similarity = similarity

In [15]:
with open(input_file, 'r') as in_f:
    lines = []
    for line in in_f:
        lines.append(line)

title = lines[0];
del lines[0:2]
comments = []

# split tree level from content
for line in lines:
  splitted_sentence = line.split('. ', 1)
  level = splitted_sentence[0]
  content = splitted_sentence[1]

  # separate Pro or Con from the actual comment
  sentiment = "Neutral"
  if "Pro:" in content or "Con: " in content:
    sentiment = content[0:3]
    content = content[5:]

  # delete links in the content
  content = re.sub(r'http\S+', '', content)
  # delete () and []
  content = content.replace('[','').replace(']','').replace('(','').replace(')','').replace('\n',"")

  c = Comment(level, content, sentiment)
  comments.append(c)

#print(comments[1])

Encoding of user comment and comments in discussion txt file + loop to calculate cos similarity

In [16]:
user_comment = "The PS5 is better because it has many more games and is used by more people."
embedding2 = model.encode(user_comment, convert_to_tensor=True)

for c in comments:
  embedding1 = model.encode(c.content, convert_to_tensor=True)
  c.setSimilarity(util.pytorch_cos_sim(embedding1, embedding2).item())

comments.sort(key=lambda x: x.similarity, reverse=True)

print("You should answer to one of these comments:")

for i in range(3):
    print(comments[i])

#print("similarity", comments[0].similarity)


You should answer to one of these comments:
1.4 The PS5 has a better lineup of games that will be released in the future. [Con]
1.3.1 The PS5 offers more exclusive games than the Series X/S. [Con]
1.1.2 This might indicate that the PS5 is worth more since it is a better console. [Con]
